# Coursera Capstone Week 4 and Week 5

## Peer-graded Assignment Week 4

### Description of the Problem and a Discussion of the Background
We want to open a restaurant in Berlin.

The Problem is that we do not know Berlin very well.

The two main question we need to answer in order to open the restaurant are:

- In which city district do we want to open the reatsurant
- What kind open restaurant shall we open

The criteria we want to use to make our decision should be the already existing retaurants around us.
We want that

- We have already some restaurants around us. This will help as customer already come to this area in order to eat.
    - So we want not to sit in the middle of nowhere wihtout any customers in our area.
- But to ensure that these customer will visit our restaurant, we need to esnure that ensure that our restaurant offers something special
    - So we want not be like our competitors around us.

### Description of the Data and how it will be used to solve the Problem
The data we use are Foursquare restaurant information.

For every city district, we will analyze where reaturants are what they offer.

Based on these information we will do clsuter analysis to answer both of our questions.

We will primarliy use
- Ratings
- Price Tier
- Location Data
- Category(Like "Café" or "Restaurant")


## Peer-graded Assignment Week 5

### Introduction, Data and Methodology 
As outlined in the section above we want to analyze the city of Berlin conerning restaurants.
- The aim of this analysis is to get an idea where it is would be a good idea to open a restaurant and what the restaurants could/should offer.
    - Anyone who is planning to open a restaurant could use this analysis.
- For that purpose we will analyze the ditrubition of restaurant across Berlin.
    - We will analyze their location.
    - We will analyze their characteristics like rating, restaurant category or price tier.
    - the data will come from the Foursaqure API.
- To peform this analysis we will
    - First get a general idea on how restaurants are distributed across Berlin by counting them per city dirtrict.
    - Then we will visuellize them across a map.
    - To get a further idea on how to sepearate the restaurants we will do a cluster analysis based on their location and visualize it.
    - Finally we will do a cluster analysis based on their characteristics.
    - Comments what is done can be found across the report.
- Presentation of results, the discussion and conclusion will be done at the and of the report.

Import Packages

In [1]:
import json
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
from pandas import json_normalize
import folium


Define API Crentials

In [2]:
CLIENT_ID = 'ODJR2W3NL0TMJO0DI4QCG1SXZZW3IPKY3XSUWYT43TUOZGKA' # your Foursquare ID
CLIENT_SECRET = 'DQCAUYRB1WOPA2NXKTMG34EJZJOUJVUIPLCS2PCLTGYWZJR2' # your Foursquare Secret
ACCESS_TOKEN = 'XUEI0DWZO0CVQZSHPVG5VUFGITNVJXNMTXOWOS4HHZMGG5LR' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
QUERY='restaurant',
radius = 6000

Define Berlin city district Geo coordinates

In [3]:
Berlin_Mitte='52.537055924203436, 13.359776678879731'
Berlin_Friedrichshain_Kreuzberg='52.49900422191546, 13.433998076851596'
Berlin_Pankow='52.59224249718644, 13.427104282715117'
Berlin_Charlottenburg_Wilmersdorf='52.51727018530877, 13.301162121534173'
Berlin_Spandau='52.53504915864648, 13.198473958050107'
Berlin_Steglitz_Zehlendorf='52.42993308479591, 13.192950967111388'
Berlin_Tempelhof_Schöneberg='52.469877080788216, 13.38337511047168'
Berlin_Neukölln='52.47402336043585, 13.446055750573192'
Berlin_Treptow_Köpenick='52.44299312491409, 13.5787488401761'
Berlin_Marzahn_Hellersdorf='52.523293403088495, 13.57907550764023'
Berlin_Lichtenberg='52.53262272849455, 13.50372688068511'
Berlin_Reinickendorf='52.573552344819646, 13.356169143154665'

city_district=[
    ('Berlin_Mitte',Berlin_Mitte),
    ('Berlin_Friedrichshain_Kreuzberg',Berlin_Friedrichshain_Kreuzberg),
    ('Berlin_Pankow',Berlin_Pankow),
    ('Berlin_Charlottenburg_Wilmersdorf',Berlin_Charlottenburg_Wilmersdorf),
    ('Berlin_Spandau',Berlin_Spandau),
    ('Berlin_Steglitz_Zehlendorf',Berlin_Steglitz_Zehlendorf),
    ('Berlin_Tempelhof_Schöneberg',Berlin_Tempelhof_Schöneberg),
    ('Berlin_Neukölln',Berlin_Neukölln),
    ('Berlin_Treptow_Köpenick',Berlin_Treptow_Köpenick),
    ('Berlin_Marzahn_Hellersdorf',Berlin_Marzahn_Hellersdorf),
    ('Berlin_Lichtenberg',Berlin_Lichtenberg),
    ('Berlin_Reinickendorf',Berlin_Reinickendorf)
]

In [4]:
column_names=['district', 'coordinates']
city_district_dataframe=pd.DataFrame(columns=column_names)
for i in city_district:
    city_district_dataframe=city_district_dataframe.append({'district': i[0], 'coordinates':i[1]}, ignore_index=True)
    
city_district_dataframe

,district,coordinates
0,Berlin_Mitte,"52.537055924203436, 13.359776678879731"
1,Berlin_Friedrichshain_Kreuzberg,"52.49900422191546, 13.433998076851596"
2,Berlin_Pankow,"52.59224249718644, 13.427104282715117"
3,Berlin_Charlottenburg_Wilmersdorf,"52.51727018530877, 13.301162121534173"
4,Berlin_Spandau,"52.53504915864648, 13.198473958050107"
5,Berlin_Steglitz_Zehlendorf,"52.42993308479591, 13.192950967111388"
6,Berlin_Tempelhof_Schöneberg,"52.469877080788216, 13.38337511047168"
7,Berlin_Neukölln,"52.47402336043585, 13.446055750573192"
8,Berlin_Treptow_Köpenick,"52.44299312491409, 13.5787488401761"
9,Berlin_Marzahn_Hellersdorf,"52.523293403088495, 13.57907550764023"


Get the number of retaurants for each city district.
- This is our first call to the API.
- We put then everything into a dataframe. Each line represents a reaturant.

In [5]:
all_restaurants_data_frame = pd.DataFrame()
restaurants_per_district=[]
for i, district in enumerate(city_district):
    LL=district[1]
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&oauth_token={}&ll={}&query={}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, LL, QUERY, VERSION, radius, LIMIT)
    json = requests.get(url).json()
    restaurants_per_district.append((district[0], json['response']['groups'][0]['items']))
    restaurants=json['response']['totalResults']
    print('In city part {} are {} Restaurants.'.format(district[0], restaurants))
    city_district_dataframe.at[i, 'restaurants']=restaurants
    values=json_normalize(json['response']['groups'][0]['items'])
    all_restaurants_data_frame=all_restaurants_data_frame.append(values)
all_restaurants_data_frame.reset_index(drop=True)
all_restaurants_data_frame

In city part Berlin_Mitte are 210 Restaurants.
In city part Berlin_Friedrichshain_Kreuzberg are 235 Restaurants.
In city part Berlin_Pankow are 99 Restaurants.
In city part Berlin_Charlottenburg_Wilmersdorf are 147 Restaurants.
In city part Berlin_Spandau are 100 Restaurants.
In city part Berlin_Steglitz_Zehlendorf are 103 Restaurants.
In city part Berlin_Tempelhof_Schöneberg are 170 Restaurants.
In city part Berlin_Neukölln are 207 Restaurants.
In city part Berlin_Treptow_Köpenick are 98 Restaurants.
In city part Berlin_Marzahn_Hellersdorf are 54 Restaurants.
In city part Berlin_Lichtenberg are 164 Restaurants.
In city part Berlin_Reinickendorf are 196 Restaurants.


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,...,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.venuePage.id,venue.location.neighborhood,venue.location.crossStreet
0,e-0-50d4563ae4b0ca04f6e12d86-0,0,"[{'summary': 'This spot is popular', 'type': '...",50d4563ae4b0ca04f6e12d86,Coffee Circle Café,Lindower Str. 18,52.543813,13.367856,"[{'label': 'display', 'lat': 52.54381308480629...",930,...,Berlin,Berlin,Deutschland,"[Lindower Str. 18, 13347 Berlin]","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],552504191,NaN,NaN
1,e-0-598cc3f99411f22af3e65ca8-1,0,"[{'summary': 'This spot is popular', 'type': '...",598cc3f99411f22af3e65ca8,ernst,Gerichtstr. 54,52.544678,13.367969,"[{'label': 'display', 'lat': 52.54467761108275...",1013,...,Berlin,Berlin,Deutschland,"[Gerichtstr. 54, 13347 Berlin]","[{'id': '52e81612bcbc57f1066b79f9', 'name': 'M...",0,[],NaN,NaN,NaN
2,e-0-51dadde3498e62a20adec62e-2,0,"[{'summary': 'This spot is popular', 'type': '...",51dadde3498e62a20adec62e,Göttlich Essen & Trinken,Tegeler Str. 23,52.541904,13.357131,"[{'label': 'display', 'lat': 52.54190361750728...",568,...,Berlin,Berlin,Deutschland,"[Tegeler Str. 23, Berlin]","[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",0,[],NaN,NaN,NaN
3,e-0-4e44c2802271bdbcf6788d0c-3,0,"[{'summary': 'This spot is popular', 'type': '...",4e44c2802271bdbcf6788d0c,Falafel Dream 2010,Müllerstr. 14a,52.543185,13.364903,"[{'label': 'display', 'lat': 52.54318461520355...",765,...,Berlin,Berlin,Deutschland,"[Müllerstr. 14a, 13347 Berlin]","[{'id': '4bf58dd8d48988d10b941735', 'name': 'F...",0,[],NaN,Leopoldplatz,NaN
4,e-0-4e2ab436d22d3f83c8867ff2-4,0,"[{'summary': 'This spot is popular', 'type': '...",4e2ab436d22d3f83c8867ff2,Güllü Lahmacun,Perleberger Str. 56,52.532217,13.350061,"[{'label': 'display', 'lat': 52.53221740861135...",850,...,Berlin,Berlin,Deutschland,"[Perleberger Str. 56, 10559 Berlin]","[{'id': '4f04af1f2fb6e1c99f3db0bb', 'name': 'T...",0,[],NaN,Moabit,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,e-0-4b178156f964a52004c623e3-95,0,"[{'summary': 'This spot is popular', 'type': '...",4b178156f964a52004c623e3,Bekarei,Dunckerstr. 23,52.544355,13.422632,"[{'label': 'display', 'lat': 52.54435463910769...",5549,...,Berlin,Berlin,Deutschland,"[Dunckerstr. 23 (Stargarder Str.), 10437 Berlin]","[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",0,[],60206179,NaN,Stargarder Str.
96,e-0-4b18388cf964a5200acf23e3-96,0,"[{'summary': 'This spot is popular', 'type': '...",4b18388cf964a5200acf23e3,Le Midi,Greifenhagener Str. 17,52.547169,13.415258,"[{'label': 'display', 'lat': 52.54716942402415...",4961,...,Berlin,Berlin,Deutschland,"[Greifenhagener Str. 17, 10437 Berlin]","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],NaN,NaN,NaN
97,e-0-532c2a87498e8a015fcc2efb-97,0,"[{'summary': 'This spot is popular', 'type': '...",532c2a87498e8a015fcc2efb,Shan Shan Bar & Restaurant,Gleimstrasse 24,52.547525,13.411258,"[{'label': 'display', 'lat': 52.54752524656165...",4721,...,Berlin,Berlin,Deutschland,"[Gleimstrasse 24, 10437 Berlin]","[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",0,[],NaN,NaN,NaN
98,e-0-4bd17cc3046076b0b4dc7171-98,0,"[{'summary': 'This spot is popular', 'type': '...",4bd17cc3046076b0b4dc7171,Hüftengold,Oderberger Straße 27,52.540120,13.405941,"[{'label': 'display', 'lat': 52.54012017414925...",5019,...,Berlin,Berlin,Deutschland,"[Oderberger Straße 27, 10435 Berlin]","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],NaN,NaN,NaN


We use then the data to visualize everything in a map.

In [6]:
from geopy.geocoders import Nominatim
address = 'Berlin, Germany'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

venues_map = folium.Map(location=[latitude, longitude], zoom_start=11) # generate map centred around Ecco

for index, row in all_restaurants_data_frame.iterrows():
    lat=row['venue.location.lat']
    lng=row['venue.location.lng']
    label=folium.Popup(row['venue.categories'], parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(venues_map)

# display map
venues_map

In [7]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.cm as cm
import matplotlib.colors as colors

Based on the location we want to do our first cluster anaysis.

In [8]:
kclusters=5
k_means = KMeans(init="k-means++", n_clusters=kclusters, n_init=5)
cluster_dataset = StandardScaler().fit_transform(all_restaurants_data_frame[['venue.location.lat', 'venue.location.lng']])
k_means.fit(cluster_dataset)
k_means.labels_
all_restaurants_data_frame.insert(0, 'Cluster Labels', k_means.labels_)
all_restaurants_data_frame.head()

,Cluster Labels,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,...,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.venuePage.id,venue.location.neighborhood,venue.location.crossStreet
0,2,e-0-50d4563ae4b0ca04f6e12d86-0,0,"[{'summary': 'This spot is popular', 'type': '...",50d4563ae4b0ca04f6e12d86,Coffee Circle Café,Lindower Str. 18,52.543813,13.367856,"[{'label': 'display', 'lat': 52.54381308480629...",...,Berlin,Berlin,Deutschland,"[Lindower Str. 18, 13347 Berlin]","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],552504191,NaN,NaN
1,2,e-0-598cc3f99411f22af3e65ca8-1,0,"[{'summary': 'This spot is popular', 'type': '...",598cc3f99411f22af3e65ca8,ernst,Gerichtstr. 54,52.544678,13.367969,"[{'label': 'display', 'lat': 52.54467761108275...",...,Berlin,Berlin,Deutschland,"[Gerichtstr. 54, 13347 Berlin]","[{'id': '52e81612bcbc57f1066b79f9', 'name': 'M...",0,[],NaN,NaN,NaN
2,2,e-0-51dadde3498e62a20adec62e-2,0,"[{'summary': 'This spot is popular', 'type': '...",51dadde3498e62a20adec62e,Göttlich Essen & Trinken,Tegeler Str. 23,52.541904,13.357131,"[{'label': 'display', 'lat': 52.54190361750728...",...,Berlin,Berlin,Deutschland,"[Tegeler Str. 23, Berlin]","[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",0,[],NaN,NaN,NaN
3,2,e-0-4e44c2802271bdbcf6788d0c-3,0,"[{'summary': 'This spot is popular', 'type': '...",4e44c2802271bdbcf6788d0c,Falafel Dream 2010,Müllerstr. 14a,52.543185,13.364903,"[{'label': 'display', 'lat': 52.54318461520355...",...,Berlin,Berlin,Deutschland,"[Müllerstr. 14a, 13347 Berlin]","[{'id': '4bf58dd8d48988d10b941735', 'name': 'F...",0,[],NaN,Leopoldplatz,NaN
4,2,e-0-4e2ab436d22d3f83c8867ff2-4,0,"[{'summary': 'This spot is popular', 'type': '...",4e2ab436d22d3f83c8867ff2,Güllü Lahmacun,Perleberger Str. 56,52.532217,13.350061,"[{'label': 'display', 'lat': 52.53221740861135...",...,Berlin,Berlin,Deutschland,"[Perleberger Str. 56, 10559 Berlin]","[{'id': '4f04af1f2fb6e1c99f3db0bb', 'name': 'T...",0,[],NaN,Moabit,NaN


We also want to show it on the map.

In [9]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

df_nextcopy = all_restaurants_data_frame.copy()

# add markers to the map
markers_colors = []
for lat, lng, cluster, label in zip(df_nextcopy['venue.location.lat'], df_nextcopy['venue.location.lng'], df_nextcopy['Cluster Labels'], df_nextcopy['venue.categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The results will be discussed in the last section.

Now we want to use the other information about the reaurants to do a cluster analysis.

Due to the fact we have only a limited amount of requests per day, we will only use the first 3 restaurants of every city district.

In [10]:
k_means_df = pd.DataFrame()
for row in restaurants_per_district:
    ids=json_normalize(row[1])[:3]
    k_means_df=k_means_df.append(ids)
k_means_df=k_means_df.reset_index(drop=True)

In [12]:
import time
ids=k_means_df['venue.id'][:]
venues=pd.DataFrame()
for i, id in enumerate(ids):
    venue_id=ids[i]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
    result = requests.get(url).json()
    print('Requests {} was done.'.format(i))
    vanue=json_normalize(result['response']['venue'])
    venues=venues.append(vanue)
    time.sleep(2)
display(venues)

Requests 0 was done.
Requests 1 was done.
Requests 2 was done.
Requests 3 was done.
Requests 4 was done.
Requests 5 was done.
Requests 6 was done.
Requests 7 was done.
Requests 8 was done.
Requests 9 was done.
Requests 10 was done.
Requests 11 was done.
Requests 12 was done.
Requests 13 was done.
Requests 14 was done.
Requests 15 was done.
Requests 16 was done.
Requests 17 was done.
Requests 18 was done.
Requests 19 was done.
Requests 20 was done.
Requests 21 was done.
Requests 22 was done.
Requests 23 was done.
Requests 24 was done.
Requests 25 was done.
Requests 26 was done.
Requests 27 was done.
Requests 28 was done.
Requests 29 was done.
Requests 30 was done.
Requests 31 was done.
Requests 32 was done.
Requests 33 was done.
Requests 34 was done.
Requests 35 was done.
Requests 36 was done.
Requests 37 was done.
Requests 38 was done.
Requests 39 was done.
Requests 40 was done.
Requests 41 was done.
Requests 42 was done.
Requests 43 was done.
Requests 44 was done.
Requests 45 was done

KeyError: 'venue'

But before we can to that we need to prepare the data.

In [17]:
venues=venues.filter(items=['rating', 'categories', 'price.tier', 'location.lat', 'location.lng']).reset_index(drop=True)
for index, categories in enumerate(venues['categories']):
    for category in categories:        
        venues.at[index, category['shortName']]=1
display(venues.drop(columns=['categories']).fillna(0))

,rating,price.tier,location.lat,location.lng,Café,Coffee Shop,Office,Modern European,Restaurant,Falafel,...,Grocery Store,Comfort Food,Vietnamese,Gastropub,Beer Garden,Deli / Bodega,Sushi,Japanese,Greek,Pub
0,9.5,2.0,52.543813,13.367856,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9.4,0.0,52.544678,13.367969,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8.6,1.0,52.541904,13.357131,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8.7,1.0,52.543185,13.364903,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8.6,1.0,52.532217,13.350061,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,9.2,1.0,52.497704,13.437417,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,9.0,1.0,52.499129,13.430566,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,9.2,2.0,52.502144,13.431607,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8.9,1.0,52.498255,13.430397,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,8.8,2.0,52.499561,13.432569,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
kclusters=4
k_means = KMeans(init="k-means++", n_clusters=kclusters, n_init=2)
cluster_dataset = StandardScaler().fit_transform(venues.drop(columns=['categories','location.lat', 'location.lng']).fillna(0))
k_means.fit(cluster_dataset)
k_means.labels_
venues.insert(0, 'Cluster Labels', k_means.labels_)
venues.head()

,Cluster Labels,rating,categories,price.tier,location.lat,location.lng,Café,Coffee Shop,Office,Modern European,...,Grocery Store,Comfort Food,Vietnamese,Gastropub,Beer Garden,Deli / Bodega,Sushi,Japanese,Greek,Pub
0,3,9.5,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",2.0,52.543813,13.367856,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,9.4,"[{'id': '52e81612bcbc57f1066b79f9', 'name': 'M...",NaN,52.544678,13.367969,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,8.6,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",1.0,52.541904,13.357131,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,8.7,"[{'id': '4bf58dd8d48988d10b941735', 'name': 'F...",1.0,52.543185,13.364903,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,8.6,"[{'id': '4f04af1f2fb6e1c99f3db0bb', 'name': 'T...",1.0,52.532217,13.350061,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
map_clusters2 = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, cluster, label in zip(venues['location.lat'], venues['location.lng'], venues['Cluster Labels'], venues['Cluster Labels']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters2)
       
map_clusters2

### Results

In [28]:
print('Cluster 0')
display(venues.loc[venues['Cluster Labels'] == 0])

print('Cluster 1')
display(venues.loc[venues['Cluster Labels'] == 1])

print('Cluster 2')
display(venues.loc[venues['Cluster Labels'] == 2])

print('Cluster 3')
display(venues.loc[venues['Cluster Labels'] == 3])

Cluster 0


,Cluster Labels,rating,categories,price.tier,location.lat,location.lng,Café,Coffee Shop,Office,Modern European,...,Grocery Store,Comfort Food,Vietnamese,Gastropub,Beer Garden,Deli / Bodega,Sushi,Japanese,Greek,Pub
4,0,8.6,"[{'id': '4f04af1f2fb6e1c99f3db0bb', 'name': 'T...",1.0,52.532217,13.350061,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0,9.2,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",2.0,52.502144,13.431607,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0,8.8,"[{'id': '4bf58dd8d48988d150941735', 'name': 'S...",2.0,52.499561,13.432569,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,0,8.8,"[{'id': '4bf58dd8d48988d10b941735', 'name': 'F...",1.0,52.512515,13.305099,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,0,8.0,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",2.0,52.532119,13.198026,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,0,8.1,"[{'id': '4bf58dd8d48988d10d941735', 'name': 'G...",2.0,52.538792,13.206109,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,0,7.8,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",1.0,52.530948,13.196511,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,0,8.0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",1.0,52.524376,13.193197,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,0,8.1,"[{'id': '53d6c1b0e4b02351e88a83d2', 'name': 'T...",NaN,52.433800,13.211532,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,0,7.7,"[{'id': '4bf58dd8d48988d1c7941735', 'name': 'S...",1.0,52.433351,13.189924,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Cluster 1


,Cluster Labels,rating,categories,price.tier,location.lat,location.lng,Café,Coffee Shop,Office,Modern European,...,Grocery Store,Comfort Food,Vietnamese,Gastropub,Beer Garden,Deli / Bodega,Sushi,Japanese,Greek,Pub
19,1,8.8,"[{'id': '5283c7b4e4b094cb91ec88d7', 'name': 'K...",NaN,52.513783,13.317013,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Cluster 2


,Cluster Labels,rating,categories,price.tier,location.lat,location.lng,Café,Coffee Shop,Office,Modern European,...,Grocery Store,Comfort Food,Vietnamese,Gastropub,Beer Garden,Deli / Bodega,Sushi,Japanese,Greek,Pub
1,2,9.4,"[{'id': '52e81612bcbc57f1066b79f9', 'name': 'M...",NaN,52.544678,13.367969,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,8.6,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",1.0,52.541904,13.357131,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2,9.2,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",1.0,52.497704,13.437417,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2,9.0,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",1.0,52.499129,13.430566,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2,8.9,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",1.0,52.498255,13.430397,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,2,8.2,"[{'id': '55a5a1ebe4b013909087cb7f', 'name': 'T...",NaN,52.586480,13.401997,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,2,8.6,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",1.0,52.603944,13.397227,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,2,8.9,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",1.0,52.569593,13.410874,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,2,8.2,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",1.0,52.578127,13.404077,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,2,8.1,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",1.0,52.580399,13.400898,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Cluster 3


,Cluster Labels,rating,categories,price.tier,location.lat,location.lng,Café,Coffee Shop,Office,Modern European,...,Grocery Store,Comfort Food,Vietnamese,Gastropub,Beer Garden,Deli / Bodega,Sushi,Japanese,Greek,Pub
0,3,9.5,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",2.0,52.543813,13.367856,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,8.7,"[{'id': '4bf58dd8d48988d10b941735', 'name': 'F...",1.0,52.543185,13.364903,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,3,8.5,"[{'id': '4bf58dd8d48988d179941735', 'name': 'B...",2.0,52.439531,13.214490,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,3,8.8,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",2.0,52.472158,13.442507,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Disussion

As it seem our charcteristics clsuter resemble the following categories
- Cluster 0 = Food - 48 entries (all kinds or restaurant) and often Low Rating
- Clsuter 1 = no data available - 1 entries
- Cluster 2 = Category: Café and Price Tier: 1 - 48 entries
- Cluster 3 = Category: Coffee Shop and Price Tier: 2 - 4 entries

It seems most of the restaurants are Cafés or Coffee Shops.

Our location Cluster show the distribution of the restaurants.

The highest density of restaurants is in the center of Berlin given with the Red Cluster

### Conclusion
Where to open a restaurant:
- We wanted an area that already has a lot of restaurants. This way a lot of customers are already in this area and want to eat in a restaurants.
- Therefore we suggest to open our restaurant in the Red area.

What kind of restaurant:
- To be able to compete with other resturants, we wanted to open a restaurant with their own characteristics.
    - Therefore we should definetely not open a Café/Coffee Shop.
- As there is no Food category which is overrepresented(Grocery Store, Comfort Food, Vietnamese, Gastropub, Beer Garden, Deli / Bodega, Sushi, Japanese, Greek Pub), we suggest to open
    - a food restaurant
    - whith a high quality to achieve a high rating

This way we should be able to be visible to a lot of customers while still having unique characteristics.